# MCQ Generator

In [9]:
# !pip install pandas openai langchain streamlit python-dotenv PyPDF2

In [85]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_community.llms import HuggingFaceHub

import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv

import PyPDF2

In [86]:
load_dotenv() #take environment variables from .env

True

In [87]:
KEY=os.getenv("OPENAI_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACE_API_KEY")

In [88]:
model1= "gpt-3.5-turbo"
model2= "gpt-4o"
model3 = "mistralai/Mistral-7B-Instruct-v0.3"
model4 = "meta-llama/Meta-Llama-3-8B-Instruct"
model5 = "meta-llama/Meta-Llama-3-8B"

In [89]:
llm1 = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)
llm2 = ChatOpenAI(openai_api_key=KEY, model_name=model2, temperature=0.7)
llm3 = HuggingFaceHub(repo_id=model3, model_kwargs={'temperature':1,'max_new_tokens': 1000})
llm4 = HuggingFaceHub(repo_id=model4, model_kwargs={'temperature':1})

c:\Users\User\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [90]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [91]:
# """
# number=5 
# subject="data science"
# tone="simple"
# """
"""
number=5 
subject="dépenses éligibles"
tone="simple"
"""

'\nnumber=5 \nsubject="dépenses éligibles"\ntone="simple"\n'

In [92]:
TEMPLATE = '''
Text:{text}
You are an expert MCQ maker. Given the above text, your job is to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conformed to the text.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}

'''

In [93]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [94]:
quiz_chain = LLMChain(llm=llm3, prompt=quiz_generator_prompt, output_key="quiz", verbose=True)

In [108]:
print(quiz_chain.run(text="Les dépenses éligibles sont les dépenses qui sont admissibles à un remboursement ou à une déduction fiscale. Ces dépenses sont généralement définies par les autorités fiscales et peuvent varier en fonction du type de dépense et de la juridiction fiscale. Par exemple, les frais de scolarité, les frais médicaux et les frais de garde d'enfants peuvent être considérés comme des dépenses éligibles dans certaines juridictions.", number=5, subject="dépenses éligibles", tone="simple", response_json=RESPONSE_JSON))
# print(quiz_chain.run(text=TEXT, number=5, subject="", tone="simple", response_json=RESPONSE_JSON))



> Entering new LLMChain chain...
Prompt after formatting:

Text:Les dépenses éligibles sont les dépenses qui sont admissibles à un remboursement ou à une déduction fiscale. Ces dépenses sont généralement définies par les autorités fiscales et peuvent varier en fonction du type de dépense et de la juridiction fiscale. Par exemple, les frais de scolarité, les frais médicaux et les frais de garde d'enfants peuvent être considérés comme des dépenses éligibles dans certaines juridictions.
You are an expert MCQ maker. Given the above text, your job is to create a quiz of 5 multiple choice questions for dépenses éligibles students in simple tone. 
Make sure the questions are not repeated and check all the questions to be conformed to the text.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make 5 MCQs.
### RESPONSE_JSON
{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice h

In [96]:
TEMPLATE2="""
You are an expert french grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert French Writer of the above quiz:
"""

In [97]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [98]:
review_chain=LLMChain(llm=llm3, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [99]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [100]:
data_file_path = "../data2.txt"

In [101]:
with open(data_file_path, "r") as file:
    TEXT = file.read()

In [102]:
print(TEXT)

Type de dépenses Principes
Frais de personnel 
Salaires chargés (dont, frais de mission) du personnel du projet (non environnés) appartenant 
aux catégories suivantes : chercheurs (post-doc inclus), ingénieurs, techniciens et autres 
personnels d'appui s'ils sont employés pour le projet.
5 https://www.green-algorithms.org/
6 Comparaison expérimentale de logiciels de mesure d'énergie : côté GPU - Inria - Institut national de recherche en sciences et technologies 
du numérique (hal.science)
7 Conformément à la réglementation européenne, les chefs de file des consortiums ne devront pas faire l’objet d’une injonction de récupération de mesure d’aide d’Etat 
émise dans une décision de la Commission européenne.
Frais connexes Montant forfaitaire des dépenses de personnel (20% des salaires chargés non environnés).
Dépenses d’achat ou de 
location de matériel 
informatique de calcul, de 
stockage
Dans la mesure où et aussi longtemps qu'ils sont utilisés pour le projet, lorsque ces instruments 

In [103]:
# Serilize the python dictionnary into a json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [104]:
NUMBER=5 
SUBJECT="Generative AI model architectures and how they have evolved"
SUBJECT="dépenses éligibles"
TONE="simple"

How to setup [Token Usage Tracking](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking) in LangChain:

In [105]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Type de dépenses Principes
Frais de personnel 
Salaires chargés (dont, frais de mission) du personnel du projet (non environnés) appartenant 
aux catégories suivantes : chercheurs (post-doc inclus), ingénieurs, techniciens et autres 
personnels d'appui s'ils sont employés pour le projet.
5 https://www.green-algorithms.org/
6 Comparaison expérimentale de logiciels de mesure d'énergie : côté GPU - Inria - Institut national de recherche en sciences et technologies 
du numérique (hal.science)
7 Conformément à la réglementation européenne, les chefs de file des consortiums ne devront pas faire l’objet d’une injonction de récupération de mesure d’aide d’Etat 
émise dans une décision de la Commission européenne.
Frais connexes Montant forfaitaire des dépenses de personnel (20% des salaires chargés non environnés).
Dépenses d’achat ou de 
location de matériel 
informatique de calcul, de 

In [106]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [107]:
response

{'text': "Type de dépenses Principes\nFrais de personnel \nSalaires chargés (dont, frais de mission) du personnel du projet (non environnés) appartenant \naux catégories suivantes : chercheurs (post-doc inclus), ingénieurs, techniciens et autres \npersonnels d'appui s'ils sont employés pour le projet.\n5 https://www.green-algorithms.org/\n6 Comparaison expérimentale de logiciels de mesure d'énergie : côté GPU - Inria - Institut national de recherche en sciences et technologies \ndu numérique (hal.science)\n7 Conformément à la réglementation européenne, les chefs de file des consortiums ne devront pas faire l’objet d’une injonction de récupération de mesure d’aide d’Etat \némise dans une décision de la Commission européenne.\nFrais connexes Montant forfaitaire des dépenses de personnel (20% des salaires chargés non environnés).\nDépenses d’achat ou de \nlocation de matériel \ninformatique de calcul, de \nstockage\nDans la mesure où et aussi longtemps qu'ils sont utilisés pour le projet,

In [59]:
quiz_str = response.get("quiz")

In [66]:
quiz_dict = json.loads(quiz_str)

In [67]:
quiz_dict

{'1': {'mcq': 'Which generative AI model architecture was introduced in 2013 and can encode data like an autoencoder but decode multiple new variations of the content?',
  'options': {'a': 'Generative adversarial networks (GANs)',
   'b': 'Transformers',
   'c': 'Variational autoencoders (VAEs)',
   'd': 'Diffusion models'},
  'correct': 'c'},
 '2': {'mcq': "Which generative AI model architecture works by first adding noise to the training data until it's random and unrecognizable, and then training the algorithm to iteratively diffuse the noise to reveal a desired output?",
  'options': {'a': 'Generative adversarial networks (GANs)',
   'b': 'Transformers',
   'c': 'Variational autoencoders (VAEs)',
   'd': 'Diffusion models'},
  'correct': 'd'},
 '3': {'mcq': "Which generative AI model architecture was first documented in a 2017 paper published by Ashish Vaswani and others, and is at the core of most of today's headline-making generative AI tools?",
  'options': {'a': 'Generative adv

In [61]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [62]:
quiz_table_data

[{'MCQ': 'Which generative AI model architecture was introduced in 2013 and can encode data like an autoencoder but decode multiple new variations of the content?',
  'Choices': 'a: Generative adversarial networks (GANs) | b: Transformers | c: Variational autoencoders (VAEs) | d: Diffusion models',
  'Correct': 'c'},
 {'MCQ': "Which generative AI model architecture works by first adding noise to the training data until it's random and unrecognizable, and then training the algorithm to iteratively diffuse the noise to reveal a desired output?",
  'Choices': 'a: Generative adversarial networks (GANs) | b: Transformers | c: Variational autoencoders (VAEs) | d: Diffusion models',
  'Correct': 'd'},
 {'MCQ': "Which generative AI model architecture was first documented in a 2017 paper published by Ashish Vaswani and others, and is at the core of most of today's headline-making generative AI tools?",
  'Choices': 'a: Generative adversarial networks (GANs) | b: Transformers | c: Variational au

In [65]:
df=pd.DataFrame(quiz_table_data)

In [64]:
df

,MCQ,Choices,Correct
0,Which generative AI model architecture was int...,a: Generative adversarial networks (GANs) | b:...,c
1,Which generative AI model architecture works b...,a: Generative adversarial networks (GANs) | b:...,d
2,Which generative AI model architecture was fir...,a: Generative adversarial networks (GANs) | b:...,b
3,Which generative AI model architecture excels ...,a: Generative adversarial networks (GANs) | b:...,b
4,Which generative AI model architecture is driv...,a: Generative adversarial networks (GANs) | b:...,d


In [69]:
df.to_csv("Generative_AI_Quiz.csv",index=False)

In [84]:
from datetime import datetime

datetime.now().strftime('%d-%m-%Y_%H:%M:%S')

'11-06-2024_17:02:40'